# Setup

In [1]:
import json

import pandas as pd
from google.cloud import securitycenter_v1
from tqdm import tqdm

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", None)

# Functions

In [2]:
scc_client = securitycenter_v1.SecurityCenterClient()


def get_finding_configs():
    with open("finding_configs.json", "r") as fp:
        return json.load(fp)


def get_findings(org_id):
    resp = scc_client.list_findings(
        securitycenter_v1.ListFindingsRequest(
            parent=f"organizations/{org_id}/sources/-",
            page_size=1000,
        )
    )
    return tqdm(resp, total=resp.total_size, unit="findings", desc="Loading Findings")


def get_assets(org_id):
    resp = scc_client.list_assets(
        securitycenter_v1.ListAssetsRequest(
            parent=f"organizations/{org_id}",
            page_size=1000,
        )
    )
    return tqdm(resp, total=resp.total_size, unit="assets", desc="Loading Assets")

# Load Data

In [3]:
org_id = "215365938848"
findings = list(get_findings(org_id))
assets = list(get_assets(org_id))
finding_configs = get_finding_configs()

Loading Assets: 100%|██████████████████████████████████████████████████████████| 8403/8403 [00:21<00:00, 391.83assets/s]


# Create Dataframes

In [4]:
adf = pd.DataFrame(
    [
        (
            a.asset.security_center_properties.resource_name,
            a.asset.security_center_properties.resource_type,
            a.asset.security_center_properties.resource_project,
            a.asset.security_center_properties.resource_owners,
        )
        for a in assets
    ],
    columns=["resource_name", "resource_type", "project", "owners"],
)

fdf = pd.DataFrame(
    [(f.finding.resource_name, f.finding.category, 1) for f in findings],
    columns=["resource_name", "category", "count"],
)

mdf = pd.DataFrame(
    [
        (
            resource_type,
            m["finding_type"],
            m["severity"],
            list(m["compliance_metadata"].keys()),
        )
        for m in finding_configs
        for resource_type in m["resource_types"]
    ],
    columns=["resource_type", "category", "severity", "benchmarks"],
)

In [5]:
adf.head()

,resource_name,resource_type,project,owners
0,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[]
1,//cloudresourcemanager.googleapis.com/projects/469450300351,google.cloud.resourcemanager.Project,//cloudresourcemanager.googleapis.com/projects/469450300351,[user:shubaiyer@google.com]
2,//cloudresourcemanager.googleapis.com/projects/350556640222,google.cloud.resourcemanager.Project,//cloudresourcemanager.googleapis.com/projects/350556640222,[user:bensanders@google.com]
3,//cloudresourcemanager.googleapis.com/projects/225465665268,google.cloud.resourcemanager.Project,//cloudresourcemanager.googleapis.com/projects/225465665268,[user:qxl@google.com]
4,//cloudresourcemanager.googleapis.com/projects/1021433544460,google.cloud.resourcemanager.Project,//cloudresourcemanager.googleapis.com/projects/1021433544460,[user:anniemao@google.com]


In [6]:
fdf.head()

,resource_name,category,count
0,//compute.googleapis.com/projects/es-demo-prod/regions/asia-south1/subnetworks/3759943304756592809,FLOW_LOGS_DISABLED,1
1,//compute.googleapis.com/projects/iap-managed-instance-group-fr/zones/us-central1-a/instances/3302980516996567441,DEFAULT_SERVICE_ACCOUNT_USED,1
2,//cloudresourcemanager.googleapis.com/projects/891045812741,CUSTOM_ROLE_NOT_MONITORED,1
3,//compute.googleapis.com/projects/iap-managed-instance-group-fr/zones/us-central1-a/instances/3674588702404281108,PUBLIC_IP_ADDRESS,1
4,//compute.googleapis.com/projects/in-vpc-scanning-demo/regions/us-east1/subnetworks/7896065624762490204,FLOW_LOGS_DISABLED,1


In [7]:
mdf.head()

,resource_type,category,severity,benchmarks
0,google.cloud.resourcemanager.Organization,ADMIN_SERVICE_ACCOUNT,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"
1,google.cloud.resourcemanager.Folder,ADMIN_SERVICE_ACCOUNT,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"
2,google.cloud.resourcemanager.Project,ADMIN_SERVICE_ACCOUNT,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"
3,google.cloud.resourcemanager.Project,API_KEY_APIS_UNRESTRICTED,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"
4,google.cloud.resourcemanager.Project,API_KEY_APPS_UNRESTRICTED,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"


# Merge

In [8]:
df = adf.merge(mdf, on="resource_type").merge(
    fdf, how="left", on=["resource_name", "category"]
)
df["count"] = df["count"].fillna(0).astype("int")
df

,resource_name,resource_type,project,owners,category,severity,benchmarks,count
0,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[],ADMIN_SERVICE_ACCOUNT,MEDIUM,"[CIS 1.0 Level 1, CIS 1.1 Level 1]",1
1,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[],AUDIT_LOGGING_DISABLED,LOW,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]",1
2,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[],KMS_ROLE_SEPARATION,MEDIUM,"[CIS 1.0 Level 2, CIS 1.1 Level 2, NIST, ISO]",0
3,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[],MFA_NOT_ENFORCED,HIGH,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]",1
4,//cloudresourcemanager.googleapis.com/organizations/215365938848,google.cloud.resourcemanager.Organization,,[],NON_ORG_IAM_MEMBER,HIGH,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]",1
...,...,...,...,...,...,...,...,...
14452,//compute.googleapis.com/projects/archery-range-test/zones/us-central1-b/networkEndpointGroups/7071147847689363169,google.compute.NetworkEndpointGroup,//cloudresourcemanager.googleapis.com/projects/705883220659,"[user:bensanders@google.com, user:mariussteffens@google.com]",ORG_POLICY_LOCATION_RESTRICTION,MEDIUM,[],0
14453,//compute.googleapis.com/projects/archery-range-test/zones/us-central1-c/networkEndpointGroups/7548110451310780153,google.compute.NetworkEndpointGroup,//cloudresourcemanager.googleapis.com/projects/705883220659,"[user:bensanders@google.com, user:mariussteffens@google.com]",ORG_POLICY_LOCATION_RESTRICTION,MEDIUM,[],0
14454,//compute.googleapis.com/projects/gclb-multi-service-test-app/zones/us-central1-a/networkEndpointGroups/4756782166325632152,google.compute.NetworkEndpointGroup,//cloudresourcemanager.googleapis.com/projects/68404284703,"[user:anniemao@google.com, user:menglish@google.com]",ORG_POLICY_LOCATION_RESTRICTION,MEDIUM,[],0
14455,//compute.googleapis.com/projects/css-gke-tests/regions/us-central1/backendServices/5717825367004480071,google.compute.RegionBackendService,//cloudresourcemanager.googleapis.com/projects/491846130587,[user:ashishin@google.com],ORG_POLICY_LOCATION_RESTRICTION,MEDIUM,[],0


# Export

In [9]:
df.to_csv("verdicts.csv")

# Compliance Coverage

## By Benchmark

In [10]:
(
    df.explode("benchmarks")
    .groupby("benchmarks")
    .agg(
        vuln=pd.NamedAgg(column="count", aggfunc="sum"),
        total=pd.NamedAgg(column="count", aggfunc="count"),
    )
    .assign(vuln_percent=lambda tdf: tdf["vuln"] * 100 / tdf["total"])
)

,vuln,total,vuln_percent
benchmarks,,,
CIS 1.0 Level 1,2024,3040,66.578947
CIS 1.0 Level 2,1336,3208,41.645885
CIS 1.1 Level 1,1876,2922,64.202601
CIS 1.1 Level 2,183,984,18.597561
ISO,1724,7628,22.600944
NIST,1702,7655,22.233834
PCI,1814,8003,22.666500


## Filtered by a Project

In [11]:
(
    df.query("project == '//cloudresourcemanager.googleapis.com/projects/705883220659'")
    .explode("benchmarks")
    .groupby("benchmarks")
    .agg(
        vuln=pd.NamedAgg(column="count", aggfunc="sum"),
        total=pd.NamedAgg(column="count", aggfunc="count"),
    )
    .assign(vuln_percent=lambda tdf: tdf["vuln"] * 100 / tdf["total"])
)

,vuln,total,vuln_percent
benchmarks,,,
CIS 1.0 Level 1,48,87,55.172414
CIS 1.0 Level 2,33,157,21.019108
CIS 1.1 Level 1,40,78,51.282051
CIS 1.1 Level 2,3,35,8.571429
ISO,42,218,19.266055
NIST,41,222,18.468468
PCI,45,240,18.750000


## By Resource Type

In [12]:
(
    df.groupby(["resource_type"])
    .agg(
        vuln=pd.NamedAgg(column="count", aggfunc="sum"),
        total=pd.NamedAgg(column="count", aggfunc="count"),
    )
    .assign(vuln_percent=lambda tdf: tdf["vuln"] * 100 / tdf["total"])
)

,vuln,total,vuln_percent
resource_type,,,
google.cloud.bigquery.Dataset,0,10,0.000000
google.cloud.dns.ManagedZone,2,4,50.000000
google.cloud.resourcemanager.Folder,0,42,0.000000
google.cloud.resourcemanager.Organization,5,9,55.555556
google.cloud.resourcemanager.Project,613,1326,46.229261
google.cloud.sql.Instance,13,54,24.074074
google.cloud.storage.Bucket,142,576,24.652778
google.compute.Address,0,38,0.000000
google.compute.Autoscaler,0,1,0.000000


## By Finding Category

In [13]:
(
    df.groupby(["category"])
    .agg(
        vuln=pd.NamedAgg(column="count", aggfunc="sum"),
        total=pd.NamedAgg(column="count", aggfunc="count"),
    )
    .assign(vuln_percent=lambda tdf: tdf["vuln"] * 100 / tdf["total"])
)

,vuln,total,vuln_percent
category,,,
ADMIN_SERVICE_ACCOUNT,5,58,8.620690
API_KEY_APIS_UNRESTRICTED,6,51,11.764706
API_KEY_APPS_UNRESTRICTED,14,51,27.450980
API_KEY_EXISTS,15,51,29.411765
API_KEY_NOT_ROTATED,14,51,27.450980
AUDIT_CONFIG_NOT_MONITORED,51,51,100.000000
AUDIT_LOGGING_DISABLED,52,52,100.000000
AUTO_BACKUP_DISABLED,0,3,0.000000
AUTO_REPAIR_DISABLED,0,13,0.000000
